# Language Detection in MP3s using Librosa and Keras
* Noah Sragow
* David Haase

## Import Libraries

In [1]:
import pandas as pd
import numpy as np

import librosa

from audioutils import AudioSplitter, greek, arabic, turkish, persian, russian

source_languages = [arabic, greek, persian, russian, turkish]
ASML = AudioSplitter(source_languages)

target_dir = '/Volumes/LaCie Rose Gold 2TB/Datasets/Audio/'
seconds = 20
train_dir, test_dir = ASML.split(write_dir=target_dir, duration_s=seconds, test_split=.1)

2531 files already found in /Volumes/LaCie Rose Gold 2TB/Datasets/Audio/20_Seconds/Train_20/
Not rebuilding segments. Pass rebuild=True to force rebuild


## Load Data
* Audio files split into 10-second extracts
* 5,256 extracts created

In [3]:
def extract_linear_features(file_path):
    num_features = 40
    res_type = 'kaiser_fast'

    # handle exception to check if there isn't a file which is corrupted
    try:
      # here kaiser_fast is a technique used for faster extraction
        X, sample_rate = librosa.load(file_path, res_type=res_type)
      # we extract mfcc feature from data
        mfccs = np.mean(librosa.feature.mfcc(y=X, sr=sample_rate, n_mfcc=num_features).T,axis=0)
    except Exception as e:
        print(e, "Error encountered while parsing file: ", file_path)
        return None

    return mfccs

In [ ]:
write_dir = '/Volumes/LaCie Rose Gold 2TB/Datasets/Features/'
kwargs = {'train_dir':train_dir, 'test_dir':test_dir, 'write_dir':write_dir, 'func':extract_linear_features}

ASML.extract_features(**kwargs)

### Extract Features

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn import metrics

from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import Convolution1D, Convolution2D, MaxPooling2D
from keras.optimizers import Adam
from keras.utils import np_utils

test_size = 0.25
random_state= 23
target = Audio_10.train_df['Language']
features = Audio_10.train_df['Features']
X_train, X_test, y_train, y_test = train_test_split(features, target, test_size=test_size, random_state=random_state)

In [ ]:
X_train = np.array(X_train.tolist())
y_train = np.array(y_train.tolist())

X_test = np.array(X_test.tolist())
y_test = np.array(y_test.tolist())

lb = LabelEncoder()

y_train = np_utils.to_categorical(lb.fit_transform(y_train))
y_test = np_utils.to_categorical(lb.fit_transform(y_test))

num_labels = y_train.shape[1]
filter_size = 2
num_epochs = 100

model = Sequential()
model.add(Dense(256, input_shape=(num_features,)))
model.add(Activation('relu'))
model.add(Dropout(0.5))

model.add(Dense(256))
model.add(Activation('relu'))
model.add(Dropout(0.5))

model.add(Dense(num_labels))
model.add(Activation('softmax'))

model.compile(loss='categorical_crossentropy', metrics=['accuracy'], optimizer='adam')

history = model.fit(X_train, y_train, epochs=num_epochs, validation_data=(X_test, y_test), shuffle=False, verbose=2)